In [1]:
import pandas as pd

C:\Users\chanc\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
filecontent = ""
with open("NER.txt", "r") as file:
    for line in file:        
        filecontent += line
        
filecontent

'On February 12 1809 Nobel Prize winner Charles Darwin was born in UK. He was a naturalist, geologist and biologist. \nOn November 20 1889 Nobel Prize winner Edwin Hubble was born in US. He was an astronomer. \nOn November 7 1867 Nobel Prize winner Marie Curie was born in Poland. She was a physicist and chemist. \nOn January 8 1942 Nobel Prize winner Stephen Hawking was born in UK. He was a physicist and cosmologist. \nOn January 23 1918 Nobel Prize winner Gertrude Elion was born in US. She was a biochemist and pharmacologist.'

In [3]:
with open('NER.txt', 'r') as file:
    data = [line for line in file.readlines()]    

In [4]:
len(data)

5

In [5]:
df = pd.DataFrame(data, columns=["text"])
df

,text
0,On February 12 1809 Nobel Prize winner Charles...
1,On November 20 1889 Nobel Prize winner Edwin H...
2,On November 7 1867 Nobel Prize winner Marie Cu...
3,On January 8 1942 Nobel Prize winner Stephen H...
4,On January 23 1918 Nobel Prize winner Gertrude...


In [6]:
import spacy

In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
text = df['text'][0]
doc = nlp(text)

feature = []
for token in doc:
    feature.append({'token': token.text , 'POS': token.pos_})

In [9]:
fdf = pd.DataFrame(feature)
fdf

,token,POS
0,On,ADP
1,February,PROPN
2,12,NUM
3,1809,NUM
4,Nobel,PROPN
5,Prize,PROPN
6,winner,NOUN
7,Charles,PROPN
8,Darwin,PROPN
9,was,AUX


In [10]:
first_tokens = ['winner', 'name']
last_tokens = ['was', 'born']
pattern_winner = [[{'LOWER' : {'IN' : first_tokens}},  #preceding words  without case matching
           {'POS':'PROPN', 'OP' : '+'},                #searching for PROPN  one or more times
           {'LOWER': {'IN' : last_tokens}} ]]          #following words  without case matching

In [11]:
from spacy.matcher import Matcher

In [12]:
nlp = spacy.load('en_core_web_sm')
def get_winner (x):    
    doc = nlp(x)    
    matcher = Matcher(nlp.vocab)    
    matcher.add('matching_winner', pattern_winner)
    matches = matcher(doc)
    sub_text = ''
    if(len(matches) > 0):    
        span = doc[matches[0][1]:matches[0][2]]         
        sub_text = span.text
        tokens = sub_text.split(' ')    
        name, surname = tokens[1:-1]
    else:
        name = "dummy_name"
        surname  = "dummy_sur"
    return name, surname


In [13]:
new_columns = ['scientist name','surname']
for n,col in enumerate(new_columns):        
    df[col] = df['text'].apply(lambda x: get_winner(x.strip())).apply(lambda x:x[n])
df

,text,scientist name,surname
0,On February 12 1809 Nobel Prize winner Charles...,Charles,Darwin
1,On November 20 1889 Nobel Prize winner Edwin H...,Edwin,Hubble
2,On November 7 1867 Nobel Prize winner Marie Cu...,Marie,Curie
3,On January 8 1942 Nobel Prize winner Stephen H...,Stephen,Hawking
4,On January 23 1918 Nobel Prize winner Gertrude...,Gertrude,Elion


In [14]:
first_tokens = ['in']
last_tokens = ['.']
pattern_country = [[{'LOWER' : {'IN' : first_tokens}},
           {'POS':'PROPN', 'OP' : '+'},
           {'LOWER': {'IN' : last_tokens}} ]]

In [15]:
def get_country(x):    
    doc = nlp(x)
    matcher = Matcher(nlp.vocab) 
    matcher.add("matching_country", pattern_country)
    matches = matcher(doc)    
    sub_text = ''    
    if(len(matches) > 0):
        span = doc[matches[0][1]:matches[0][2]] 
        sub_text = span.text
        #remove punct
        sub_text = sub_text[:-1]
        tokens = sub_text.split(' ')
        country = ' '.join(tokens[1:])
    else:
        country = 'unknown'
    return country

In [16]:
for n,col in enumerate(new_columns):        
    df['country'] = df['text'].apply(lambda x: get_country(x.strip()))
df

,text,scientist name,surname,country
0,On February 12 1809 Nobel Prize winner Charles...,Charles,Darwin,UK
1,On November 20 1889 Nobel Prize winner Edwin H...,Edwin,Hubble,US
2,On November 7 1867 Nobel Prize winner Marie Cu...,Marie,Curie,Poland
3,On January 8 1942 Nobel Prize winner Stephen H...,Stephen,Hawking,UK
4,On January 23 1918 Nobel Prize winner Gertrude...,Gertrude,Elion,US


In [17]:

first_tokens = ['on']
last_tokens = ['nobel']
pattern_dob = [[{'LOWER' : {'IN' : first_tokens}},  #preceding words  without case matching
           {'POS':'PROPN'},                         #searching for PROPN
            {'POS':'NUM', 'OP' : '+'},              #searching for NUM  one or more times
           {'LOWER': {'IN' : last_tokens}} ]]          #following words  without case matching

In [18]:
def get_dob(x):       
    doc = nlp(x)    
    matcher = Matcher(nlp.vocab) 
    matcher.add("matching_dob", pattern_dob)
    matches = matcher(doc)    
    sub_text = ''    
    if(len(matches) > 0):        
        span = doc[matches[0][1]:matches[0][2]]         
        sub_text = span.text        
        sub_text = sub_text[:-1]              
        tokens = sub_text.split(' ')        
        dob = ' '.join(tokens[1:-1])        
    else:
        dob = 'unknown'
    return dob

In [19]:

for n,col in enumerate(new_columns):        
    df['dob'] = df['text'].apply(lambda x: get_dob(x.strip()))
df

,text,scientist name,surname,country,dob
0,On February 12 1809 Nobel Prize winner Charles...,Charles,Darwin,UK,February 12 1809
1,On November 20 1889 Nobel Prize winner Edwin H...,Edwin,Hubble,US,November 20 1889
2,On November 7 1867 Nobel Prize winner Marie Cu...,Marie,Curie,Poland,November 7 1867
3,On January 8 1942 Nobel Prize winner Stephen H...,Stephen,Hawking,UK,January 8 1942
4,On January 23 1918 Nobel Prize winner Gertrude...,Gertrude,Elion,US,January 23 1918


In [20]:
def getGender(x):
    if 'He' in x:
        return 'M'    
    return 'F'

In [21]:
for n,col in enumerate(new_columns):        
    df['gender'] = df['text'].apply(lambda x: getGender(x.strip()))
df

,text,scientist name,surname,country,dob,gender
0,On February 12 1809 Nobel Prize winner Charles...,Charles,Darwin,UK,February 12 1809,M
1,On November 20 1889 Nobel Prize winner Edwin H...,Edwin,Hubble,US,November 20 1889,M
2,On November 7 1867 Nobel Prize winner Marie Cu...,Marie,Curie,Poland,November 7 1867,F
3,On January 8 1942 Nobel Prize winner Stephen H...,Stephen,Hawking,UK,January 8 1942,M
4,On January 23 1918 Nobel Prize winner Gertrude...,Gertrude,Elion,US,January 23 1918,F
